<a href="https://colab.research.google.com/github/sobil-dalal/SignatureMatch/blob/main/SignatureMatchv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Package

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras import applications
from keras.models import Model
from keras.layers import Input,concatenate
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
import cv2
import keras as kr

import random

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarn

In [44]:
!pip install keras==2.0.8

     |████████████████████████████████| 276kB 2.7MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [45]:
!pip install tensorflow==1.3.0

     |████████████████████████████████| 43.6MB 89kB/s 
     |████████████████████████████████| 1.6MB 37.5MB/s 
     |████████████████████████████████| 890kB 39.1MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=03990d295c8e9bc6252e7580a82b41f656fb419a6090d88241cab356586c79b8
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.2.1
    Uninstalling bleach-3.2.1:
      Successfully uninstalled bleach-3.2.1
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
print(tf.__version__)
print(kr.__version__)

1.3.0
2.0.8


# <font color='blue'>Load Dataset</font>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/"

In [12]:
os.mkdir("SignatureMatch")
os.chdir("/content/SignatureMatch")
os.getcwd()

'/content/SignatureMatch'

In [13]:
!kaggle datasets download -d divyanshrai/handwritten-signatures

 99% 365M/370M [00:02<00:00, 155MB/s]
100% 370M/370M [00:02<00:00, 153MB/s]


In [14]:
#os.chdir("/content")
#!rm -rf SignatureMatch
#os.getcwd()

In [15]:
!unzip /content/SignatureMatch/handwritten-signatures.zip

Archive:  /content/SignatureMatch/handwritten-signatures.zip
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100001.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100002.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100003.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100004.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100005.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100006.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100007.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100008.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100009.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100010.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100011.png  
  inflating: Dataset_Signature_Final/Dataset/dataset1/forge/02100012.png  
  inflating: Dataset_Signature_Final/Da

In [17]:
!rm /content/SignatureMatch/handwritten-signatures.zip

# <font color='blue'>Creating the dataset for modeling</font>

In [31]:
os.listdir("/content/SignatureMatch/sample_signature/sample_Signature/")

['genuine', 'forged', 'README.txt.txt']

In [3]:
gen="/content/SignatureMatch/sample_signature/sample_Signature/genuine/"
forg="/content/SignatureMatch/sample_signature/sample_Signature/forged/"

In [4]:
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m

def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m

In [5]:
def generator():
    for i in range(1,31):
        if(i<10):
            anc,neg,pos=getfiles(float("0.00"+str(i)),gen,forg)
        else:
            anc,neg,pos=getfiles(float("0.0"+str(i)),gen,forg)
        for i in range(len(neg)):
            for j in range(len(pos)):
                anchor=returnimages(gen,anc)
                positive=returnimages(gen,pos[j])
                negative=returnimages(forg,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],[0])

## Defining loss functions

In [6]:
def triplet_loss(y_true, y_pred):
    alpha = 0.5 #whatt is the alpha
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]# why do we take the same cell twice?
    #i mean, 0:512, 512:1024 overlap while 0:511, 512:1023 from the testing file didn't
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

# <font color='blue'>Training Model</font>

## Training triplet loss

In [7]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')

In [8]:
for layer in model.layers[:15]:
    #why are just the top 15 not trainable
    layer.trainable = False

In [9]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

#what exactly are these? I heard about siamese nets, but what about this anchor out ?

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [10]:
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [11]:
model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [43]:
generator()

<generator object generator at 0x7f795805ba40>

In [12]:
for x in range(2):
    model.fit_generator(generator(),steps_per_epoch=200,epochs=3)

Epoch 1/3


AttributeError: ignored